In [1]:
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands

In [2]:
etf_list = ['QQQ', 'GLD', 'IWM', 'XLE', 'XLF', 'TLT', 'SPY']
etfs = {}

for etf in etf_list:
    df = pd.read_csv(f"../data/{etf}_with_regime.csv", index_col=0, parse_dates=True)
    df.rename(columns={etf: 'Close'}, inplace=True)  # standardize
    df.dropna(inplace=True)

    # Basic Features
    df[f'{etf}_MACD'] = MACD(df['Close']).macd()
    df[f'{etf}_RSI'] = RSIIndicator(df['Close']).rsi()
    df[f'{etf}_Bollinger_Width'] = BollingerBands(df['Close']).bollinger_wband()
    df[f'{etf}_Sharpe'] = (df['Close'].pct_change().mean() / df['Close'].pct_change().std()) * np.sqrt(252)

    etfs[etf] = df[[f'{etf}_MACD', f'{etf}_RSI', f'{etf}_Bollinger_Width', f'{etf}_Sharpe', 'Regime']]

In [3]:
from functools import reduce

In [8]:
# Start with QQQ as base
master_df = etfs['QQQ'].copy()

# Merge other ETFs' data with unique suffixes to avoid column name clashes
for etf in etf_list:
    if etf == 'QQQ':
        continue

    # Add ETF-specific suffix to all columns before joining
    suffix = f"_{etf}"
    etf_df = etfs[etf].add_suffix(suffix)

    # Join using inner join on index (e.g. date)
    master_df = master_df.join(etf_df, how='inner')

# Add QQQ future return direction as binary target
master_df['QQQ_Return'] = master_df['QQQ_MACD'].shift(-1)  # can change to 'Close' or another signal
master_df['Target'] = (master_df['QQQ_Return'] > 0).astype(int)

# Drop rows with NaNs introduced by shifting/joining
master_df.dropna(inplace=True)

# Save to CSV
master_df.to_csv("../data/qqq_supervised.csv")
print(" Saved: qqq_supervised.csv with cross-ETF features")


 Saved: qqq_supervised.csv with cross-ETF features
